In [71]:
import pandas as pd
import datetime
import numpy as np
import os.path
import wget
from zipfile import ZipFile
import shutil
import re
from multiprocessing import Pool
import multiprocessing as mp

In [72]:
## Variaveis globais

#Pega data corrente e ano corrente
date = datetime.date.today()
year = date.year

# ano inicial pra coleta dos dados
year_0 = 2010 
year_0_tri = year-2 # ano inicial pra coletoa dos dados trimestrais

In [3]:
# baixa todos os dados se não estiverem presentes, retorna uma lista de nomes de arquivos baixados
def baixa_dados(year_0=year_0, year=year, year_0_tri=year_0_tri):
    # URL's onde estão os arquivos dfp e itr na CVM
    url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'
    url_base_fre = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FRE/DADOS/'
    url_base_tri = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
    url_base_fca = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FCA/DADOS/"

    # Cria uma lista com os nomes dos arquivos de dados anuais (dfp) da CVM
    arquivos_zip = []
    for ano in range(year_0, year + 1):
        arquivos_zip.append(f'dfp_cia_aberta_{ano}.zip')

    # baixa os arquivos dfp
    arquivos_zip_dfp_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_dfp_baixados.append(arq)
        else:
            try:
                wget.download(url_base + arq, out=f"./Data/{arq}")
                arquivos_zip_dfp_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                pass

    # Cria lista de arquivos ITR (trimestrais) pra baixar
    arquivos_zip = []
    for ano in range(year_0_tri, year + 1):
        arquivos_zip.append(f'itr_cia_aberta_{ano}.zip')

    # baixa os arquivos ITR (dados trimestrais) se não estiverem presentes
    arquivos_zip_itr_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_itr_baixados.append(arq)
        else:
            try:
                wget.download(url_base_tri + arq, out=f"./Data/{arq}")
                arquivos_zip_itr_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                pass

    # baixa arquivo FCA, que contém os tickers

    # Cria lista de arquivos FCA  pra baixar. Este aqui só precisa baixar o do último ano disponível
    # Se não tiver do ano atual baixa o do ano passado
    arquivos_zip = []
    for ano in [year, year - 1]:
        arquivos_zip.append(f'fca_cia_aberta_{ano}.zip')

    arquivos_zip_fca_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_fca_baixados.append(arq)
            break
        else:
            try:
                wget.download(url_base_fca + arq, out=f"./Data/{arq}")
                arquivos_zip_fca_baixados.append(arq)
                break # se conseguiu baixar 1 já sai do loop
            except:
                print(f"falha ao baixar {arq}")
    
    # baixa arquivo FRE, que contém o número total de ações
    
    arquivos_zip = []
    for ano in range(year-3, year+1):
        arquivos_zip.append(f'fre_cia_aberta_{ano}.zip')

    arquivos_zip_fre_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_fre_baixados.append(arq)
        else:
            try:
                wget.download(url_base_fre + arq, out=f"./Data/{arq}")
                arquivos_zip_fre_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                
    return (arquivos_zip_dfp_baixados, arquivos_zip_itr_baixados, arquivos_zip_fca_baixados, arquivos_zip_fre_baixados)

In [4]:
def processa_dados(year_0=year_0, year=year, year_0_tri=year_0_tri):
    print("Processando dados")
    
    # DFP - Dados anuais
    # junta todos os anos de dados anuais em um arquivo só de cada tipo
    nomes = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DRE_con', 'DRE_ind',
             'DFC_MD_con', 'DFC_MI_con', 'DFC_MD_ind', 'DFC_MI_ind']
    for nome in nomes:
        arquivo_bp = pd.DataFrame()
        for ano in range(year_0, year):
            try:
                arquivo_bp = pd.concat([arquivo_bp,
                                        pd.read_csv(f'./Data/temp/dfp_cia_aberta_{nome}_{ano}.csv', sep=';',
                                                    decimal=',', encoding='ISO-8859-1')])
            except:
                print(f"falha em {nome}")
                pass
        arquivo_bp.to_csv(f'./Data/processados/dfp_cia_aberta_{nome}_{year_0}-{year}.csv', index=False, encoding="latin1")
        print(f'DFP {nome} processado')
    
    # ITR - Dados trimestrais
    # junta todos os anos de dados trimestrais em um arquivo só de cada tipo
    nomes_tri = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DRE_con', 'DRE_ind',
                 "DFC_MD_con", "DFC_MI_con", "DFC_MD_ind", "DFC_MI_ind"]
    for nome in nomes_tri:
        arquivo_bp = pd.DataFrame()
        for ano in range(year_0_tri, year + 1):
            try:
                arquivo_bp = pd.concat([arquivo_bp,
                                        pd.read_csv(f'./Data/temp/itr_cia_aberta_{nome}_{ano}.csv', sep=';',
                                                    decimal=',', encoding='ISO-8859-1')])
            except:
                print(f"falha em itr_cia_aberta_{nome}_{ano}")
                pass
        arquivo_bp.to_csv(f'./Data/processados/itr_cia_aberta_{nome}_{year_0_tri}-{year}.csv', index=False, encoding="latin1")
        print(f'ITR {nome} processado')
    
    # FRE - Contém q qtd de ações das empresas
    
    # cria uma lista com todos os arquivos fre
    arq_fre = [item for item in os.listdir("./Data/temp") if item.startswith("fre_cia_aberta_distribuicao_capital_2")]
    fre = pd.DataFrame()

    # concatena tudo
    for arq in arq_fre[::-1]:
        fre = pd.concat([fre, pd.read_csv(f"./Data/temp/{arq}", sep=";")], axis=0)

    # joga fora as linhas com 0
    fre = fre[fre["Quantidade_Total_Acoes_Circulacao"] != 0]

    # ordena pelo mais recente, retira as duplicatas e pega só as colunas que interessam
    fre = fre.sort_values(by=["Data_Referencia", "Versao"], ascending=False).drop_duplicates(["CNPJ_Companhia"], keep="first")[["CNPJ_Companhia", "Quantidade_Total_Acoes_Circulacao"]]

    # salva em um csv
    fre.to_csv("./Data/processados/qtd_acoes.csv", encoding="latin1", index=False)
    #fre.set_index("CNPJ_Companhia", inplace=True)
    print("FRE processado")
    
    # arquivo FCA - Contém tickers das empresas
    
    # pega o nome do arquivo fca que contem os tickers
    arq_fca = [item for item in os.listdir("./Data/temp") if item.startswith("fca_cia_aberta_valor_mobiliario")]
    arq_fca = arq_fca[0]

    # copia o arquivo fca pra pasta processados
    shutil.copyfile(f"./Data/temp/{arq_fca}", f"./Data/processados/fca.csv")
    print("FCA processado")
        
    print("Arquivos processados")

In [5]:
# só extrai o arquivo dado como parâmetro para a pasta ./Data/temp
def extrai_arquivo(arq):
    try:
        ZipFile(f"./Data/{arq}", 'r').extractall('./Data/temp')
        print(f"{arq} extraido")
    except:
        print(f"erro ao extrair {arq}")

# extrai uma lista de arquivos com paralelismo. não roda no jupyter

def extrai_paralelo(arquivos_para_extrair):

    #  descobre quantas threads tem o CPU atual e cria uma pool para extrair os dados com paralelismo
    procs = mp.cpu_count()
    pool = Pool(procs)

    # cria uma lista de pools para depois dar get
    lista_pool = []
    for arq in arquivos_para_extrair:
        lista_pool.append(pool.apply_async(extrai_arquivo, args=(arq,)))
    print("Extraindo")
    output = [item.get() for item in lista_pool]
    return(output)

In [6]:
# faz o usuario escolher uma empresa
def escolhe_empresa(lista_empresas):
    while True:
        busca = input("Entre com o ticker da empresa sem o número: ")
        empresa = lista_empresas[lista_empresas['TICKER'].str.contains(busca, case=False)]
        print(empresa[["DENOM_CIA", "CD_CVM", "TICKER"]])
        ok = input("A empresa desejada está na primeira linha? (y/n) ")
        if ok == "y":
            return(empresa)

In [7]:
# retorna um df com CNPJ razao social e ticker de todas as empresas listadas em bolsa
def cria_lista_empresas(year_0=year_0, year=year, year_0_tri=year_0_tri):

    #cria lista de empresas. Concatena consolidado e individual
    lista_empresas = pd.read_csv(f"./Data/processados/dfp_cia_aberta_BPA_con_{year_0}-{year}.csv", header=0, sep=",", encoding="latin1")[["DENOM_CIA", 'CD_CVM', "CNPJ_CIA"]].drop_duplicates()
    lista_empresas["tipo"] = "con"
    ind = pd.read_csv(f"./Data/processados/dfp_cia_aberta_BPA_ind_{year_0}-{year}.csv", header=0, sep=",", encoding="latin1")[["DENOM_CIA", 'CD_CVM', "CNPJ_CIA"]].drop_duplicates()
    ind["tipo"] = "ind"
    lista_empresas = pd.concat([lista_empresas, ind], ignore_index=True).drop_duplicates(keep="first")
    lista_empresas.set_index("CNPJ_CIA", inplace=True)

    # coloca tickers do arquivo fca
    lista_empresas = lista_empresas.join(pd.read_csv(f"./Data/processados/fca.csv", header=0, sep=";", encoding="latin1").set_index("CNPJ_Companhia")["Codigo_Negociacao"].dropna())

    # pega tickers da B3 tbm pq na CVM é incompleto.
    # extraí de https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm
    # no futuro seria bom automatizar isso com um webscraper
    b3 = pd.read_csv("./Data/empresas_listadas_b3.csv", encoding="latin1", sep=";")[["Razão Social","Código"]]

    # Cria a coluna CNPJ com os dados do index
    lista_empresas = lista_empresas.rename_axis('CNPJ').reset_index()

    # Junta os dados da b3 no arquivo original
    lista_empresas = lista_empresas.merge(b3, left_on="DENOM_CIA", right_on="Razão Social", how="left").drop(columns=["Razão Social"])
    
    # Dá replace em todas as linhas de "Codigo_Negociacao" que não obedecem o padrão "ABCD1" ou "ABCD11"
    lista_empresas["Codigo_Negociacao"].replace(to_replace= r"^(?!\D{4}\d{1,2}$).*", value= np.nan, regex=True, inplace=True)

    # Pega só as 4 primeiras letras da coluna "Codigo_Negociacao"
    lista_empresas["TICKER"] = lista_empresas["Codigo_Negociacao"].str[:4]
    lista_empresas.drop(columns=["Codigo_Negociacao"], inplace=True)
    lista_empresas.drop_duplicates(inplace=True)

    # Dá replace nos NaN da CVM com o ticker do site da B3
    lista_empresas["TICKER"].fillna(lista_empresas["Código"], inplace=True)

    # Joga fora a coluna com tickers da B3
    lista_empresas.drop(columns=["Código"], inplace=True)
    
    # joga fora o que não tiver ticker
    lista_empresas.dropna(inplace=True)
    
    #deixa o index como o CNPJ
    lista_empresas.set_index("CNPJ")
    
    # tira linhas duplicadas (tem que ter CNPJ, CD_CVM e TICKER iguais)
    lista_empresas.drop_duplicates(["CNPJ", "CD_CVM", "TICKER"], inplace=True)
    
    # muda o index pro CNPJ pra poder dar join
    lista_empresas.set_index("CNPJ", inplace=True)

    # pega a quantidade de ações em circulação e tira as empresas que não tem nenhuma. isso limpa bem os dados
    fre = pd.read_csv("./Data/processados/qtd_acoes.csv").set_index("CNPJ_Companhia")
    lista_empresas = lista_empresas.join(fre).dropna()
    
    #salva em um csv
    lista_empresas.to_csv("./Data/processados/lista empresas.csv", encoding="latin1", index=False)
    
    return(lista_empresas)

In [8]:
# funcao colocar lucros ultimos 10 anos na lista_empresas e calcular indicadores
def Coloca_lucro(lista_empresas, year_0=year_0, year=year, year_0_tri=year_0_tri):

    # processa separado pra empresas consolidadas e individuais
    lista_empresas_con = lista_empresas[lista_empresas["tipo"] == "con"]
    
    # cria df dre pros dados consolidado
    tipo= "con"
    dre = pd.concat([pd.read_csv(f'./Data/processados/dfp_cia_aberta_DRE_{tipo}_{year_0}-{year}.csv', encoding="latin1"),
                 pd.read_csv(f'./Data/processados/itr_cia_aberta_DRE_{tipo}_{year_0_tri}-{year}.csv', encoding="latin1")],
               ignore_index=True).drop_duplicates()
    dre = dre[dre["ORDEM_EXERC"] == "ÚLTIMO"]
    dre.set_index("CNPJ_CIA", inplace=True)
    # pega só o lucro
    dre = dre[(((dre.DS_CONTA == 'Lucro/Prejuízo Consolidado do Período')) | 
               (dre.DS_CONTA == 'Lucro ou Prejuízo Líquido Consolidado do Período'))].copy()
    # multiplica o valor por 1000 onde a escala é 1000
    dre.loc[(dre.ESCALA_MOEDA == "MIL"), "VL_CONTA"] *= 1000
    
    
    # coloca as colunas de lucro com o nome da coluna correspondendo ao trimestre/ano e calcula o 4T
    for ano in range(year_0, year+1):
        for tri_ini, tri_fim, nome_tri in zip(["01", "04", "07", "10", "01"], ["03", "06", "09", "12", "12"], ["1T", "2T", "3T", "4T", ""]):
            lista_empresas_con = lista_empresas_con.join(dre[(dre.DT_INI_EXERC == (f"{ano}-{tri_ini}-01")) & 
                                                             (dre.DT_FIM_EXERC.str.contains(f"{ano}-{tri_fim}-3")) & 
                                                             (((dre.DS_CONTA == 'Lucro/Prejuízo Consolidado do Período')) | (dre.DS_CONTA == 'Lucro ou Prejuízo Líquido Consolidado do Período'))][
                ["VL_CONTA", "DS_CONTA"]].rename(columns={"VL_CONTA": f"{nome_tri}{ano}"}).drop("DS_CONTA", axis=1))

        lista_empresas_con[f"4T{ano}"] = lista_empresas_con[f"{ano}"] - lista_empresas_con[f"3T{ano}"] - lista_empresas_con[f"2T{ano}"] - lista_empresas_con[f"1T{ano}"]
    lista_empresas_con.dropna(axis=1, inplace=True, how="all")
    
    # tira o cnpj do index pra poder rodar em todas as linhas
    lista_empresas_con = lista_empresas_con.rename_axis('CNPJ').reset_index()
    
    # cria coluna TTM (últimos 12 meses)
    #Para todas as linhas:
    for linha in lista_empresas_con.index:
        # pega os nomes das últimas 4 colunas que são trimestrais, contém dados e são do ano atual ou do último
        ultimos_4_tri = [item for item in list(lista_empresas_con.loc[linha].dropna().index) if (bool(re.search(r"^(\dT)", item)) and ((item.endswith(str(year))) or (item.endswith(str(year-1)))))][-4:]
        # se tiver 4 trimestres, soma e põe na caoluna TTM
        if len(ultimos_4_tri) == 4:
            lista_empresas_con.loc[linha, "TTM"] = lista_empresas_con.loc[linha, ultimos_4_tri].sum()
    
    # dropa todas as colunas trimestrais
    colunas_tri = [item for item in list(lista_empresas_con.columns) if (bool(re.search(r"^(\dT)", item)))]
    lista_empresas_con.drop(columns=colunas_tri, inplace=True)

    # ############################################################################################## 
    # agora faz pro individual
    
    # processa separado pra empresas consolidadas e individuais
    lista_empresas_ind = lista_empresas[lista_empresas["tipo"] == "ind"]
    
    # cria df dre pros dados consolidado
    tipo= "ind"
    dre = pd.concat([pd.read_csv(f'./Data/processados/dfp_cia_aberta_DRE_{tipo}_{year_0}-{year}.csv', encoding="latin1"),
                 pd.read_csv(f'./Data/processados/itr_cia_aberta_DRE_{tipo}_{year_0_tri}-{year}.csv', encoding="latin1")],
               ignore_index=True).drop_duplicates()
    dre = dre[dre["ORDEM_EXERC"] == "ÚLTIMO"]
    dre.set_index("CNPJ_CIA", inplace=True)
    # pega só o lucro
    dre = dre[((dre.DS_CONTA == 'Lucro/Prejuízo do Período') | 
               (dre.DS_CONTA == 'Lucro ou Prejuízo Líquido do Período'))].copy()
    # multiplica o valor por 1000 onde a escala é 1000
    dre.loc[(dre.ESCALA_MOEDA == "MIL"), "VL_CONTA"] *= 1000
    
    # coloca as colunas de lucro com o nome da coluna correspondendo ao trimestre/ano e calcula o 4T
    for ano in range(year_0, year+1):
        for tri_ini, tri_fim, nome_tri in zip(["01", "04", "07", "10", "01"], ["03", "06", "09", "12", "12"], ["1T", "2T", "3T", "4T", ""]):
            lista_empresas_ind = lista_empresas_ind.join(dre[(dre.DT_INI_EXERC == (f"{ano}-{tri_ini}-01")) & 
                                                             (dre.DT_FIM_EXERC.str.contains(f"{ano}-{tri_fim}-")) & 
                                                             ((dre.DS_CONTA == 'Lucro/Prejuízo do Período') | ((dre.DS_CONTA == 'Lucro ou Prejuízo Líquido do Período')))][
                ["VL_CONTA", "DS_CONTA"]].rename(columns={"VL_CONTA": f"{nome_tri}{ano}"}).drop("DS_CONTA", axis=1))

        lista_empresas_ind[f"4T{ano}"] = lista_empresas_ind[f"{ano}"] - lista_empresas_ind[f"3T{ano}"] - lista_empresas_ind[f"2T{ano}"] - lista_empresas_ind[f"1T{ano}"]
    lista_empresas_ind.dropna(axis=1, inplace=True, how="all")
    
    # tira o cnpj do index pra poder rodar em todas as linhas
    lista_empresas_ind = lista_empresas_ind.rename_axis('CNPJ').reset_index()
    
    # cria coluna TTM (últimos 12 meses)
    #Para todas as linhas:
    for linha in lista_empresas_ind.index:
        # pega os nomes das últimas 4 colunas que são trimestrais, contém dados e são do ano atual ou do último
        ultimos_4_tri = [item for item in list(lista_empresas_ind.loc[linha].dropna().index) if (bool(re.search(r"^(\dT)", item)) and ((item.endswith(str(year))) or (item.endswith(str(year-1)))))][-4:]
        # se tiver 4 trimestres, soma e põe na coluna TTM
        if len(ultimos_4_tri) == 4:
            lista_empresas_ind.loc[linha, "TTM"] = lista_empresas_ind.loc[linha, ultimos_4_tri].sum()
    
    # dropa todas as colunas trimestrais
    colunas_tri = [item for item in list(lista_empresas_ind.columns) if (bool(re.search(r"^(\dT)", item)))]
    lista_empresas_ind.drop(columns=colunas_tri, inplace=True)
    
    # junta con e ind
    return(pd.concat([lista_empresas_con, lista_empresas_ind], axis=0, ignore_index=True).set_index("CNPJ").drop_duplicates())

In [9]:
# ve se o lucro dos últimos 10 anos foi maior que 0 e se o crescimento foi maior que 30%. Coloca em 2 novas colunas
# dá NaN quando não tem dados suficientes pra calcular, 1 pra quando passa no filtro e 0 pra quando não passa
def Indicadores_lucro(lista_empresas):
    # pra todas as linhas
    for linha in lista_empresas.index:
        # Faz uma lista com os últimos 10 anos que contém dados (contando com TTM)
        ultimos_10_anos = [item for item in list(lista_empresas.loc[linha].dropna().index) if (bool(re.search(r"^(\d{4})", item)) or (item == "TTM"))][-10:]
        # se tiver 10 anos de dados
        if (len(ultimos_10_anos) == 10):
            # e o lucro for positivo pra todos os elementos da linha
            if ((lista_empresas.loc[linha, ultimos_10_anos] > 0).all()):
                # a coluna "Sem prejuízo nos últimos 10 anos" recebe 1,
                lista_empresas.loc[linha, "Sem prejuízo nos últimos 10 anos"] = 1
            else:
                #  se não, 0
                lista_empresas.loc[linha, "Sem prejuízo nos últimos 10 anos"] = 0

            # Filtro de crescimento de 30% do lucro nos últimos 10 anos
            # se o lucro do ultimo ano/ 10 anos atras for maior ou igual a 30%
            if (lista_empresas.loc[linha, ultimos_10_anos[-1]]/lista_empresas.loc[linha, ultimos_10_anos[0]]) >= 1.3:
                lista_empresas.loc[linha, "Crescimento de 30% dos lucros nos últimos 10 anos"] = 1
            else:
                lista_empresas.loc[linha, "Crescimento de 30% dos lucros nos últimos 10 anos"] = 0
    return(lista_empresas)

## Main

In [10]:
# Limpeza

# Deleta o conteúdo da pasta processados se existir
lista_arquivos = os.listdir("./Data/processados/")
lista_arquivos.remove(".gitignore")

for item in lista_arquivos:
    try:
        os.remove(f"./Data/processados/{item}")
    except:
        pass

#deleta pasta temporaria se existir
try:
    shutil.rmtree(f'./Data/temp')
except:
    pass

In [11]:
# baixa os dados e coloca os nomes dos arquivos baixados em cada variavel
arquivos_dfp, arquivos_itr, arquivos_fca, arquivos_fre = baixa_dados(year_0, year, year_0_tri)
arquivos_dfp, arquivos_itr, arquivos_fca, arquivos_fre

dfp_cia_aberta_2010.zip já esta aqui
dfp_cia_aberta_2011.zip já esta aqui
dfp_cia_aberta_2012.zip já esta aqui
dfp_cia_aberta_2013.zip já esta aqui
dfp_cia_aberta_2014.zip já esta aqui
dfp_cia_aberta_2015.zip já esta aqui
dfp_cia_aberta_2016.zip já esta aqui
dfp_cia_aberta_2017.zip já esta aqui
dfp_cia_aberta_2018.zip já esta aqui
dfp_cia_aberta_2019.zip já esta aqui
dfp_cia_aberta_2020.zip já esta aqui
dfp_cia_aberta_2021.zip já esta aqui
100% [............................................................................] 324269 / 324269fre_cia_aberta_2019.zip já esta aqui
fre_cia_aberta_2020.zip já esta aqui
fre_cia_aberta_2021.zip já esta aqui
100% [..........................................................................] 9511350 / 9511350

(['dfp_cia_aberta_2010.zip',
  'dfp_cia_aberta_2011.zip',
  'dfp_cia_aberta_2012.zip',
  'dfp_cia_aberta_2013.zip',
  'dfp_cia_aberta_2014.zip',
  'dfp_cia_aberta_2015.zip',
  'dfp_cia_aberta_2016.zip',
  'dfp_cia_aberta_2017.zip',
  'dfp_cia_aberta_2018.zip',
  'dfp_cia_aberta_2019.zip',
  'dfp_cia_aberta_2020.zip',
  'dfp_cia_aberta_2021.zip',
  'dfp_cia_aberta_2022.zip'],
 ['itr_cia_aberta_2020.zip',
  'itr_cia_aberta_2021.zip',
  'itr_cia_aberta_2022.zip'],
 ['fca_cia_aberta_2022.zip'],
 ['fre_cia_aberta_2019.zip',
  'fre_cia_aberta_2020.zip',
  'fre_cia_aberta_2021.zip',
  'fre_cia_aberta_2022.zip'])

In [12]:
# extrai arquivos
arquivos_para_extrair = arquivos_dfp + arquivos_itr + arquivos_fca + arquivos_fre

# extrai sequencial
for arq in arquivos_para_extrair:
    extrai_arquivo(arq)
print("Pronto")
# extrai paralelo, não funciona no jupyter
#extrai_paralelo(arquivos_para_extrair)

dfp_cia_aberta_2010.zip extraido
dfp_cia_aberta_2011.zip extraido
dfp_cia_aberta_2012.zip extraido
dfp_cia_aberta_2013.zip extraido
dfp_cia_aberta_2014.zip extraido
dfp_cia_aberta_2015.zip extraido
dfp_cia_aberta_2016.zip extraido
dfp_cia_aberta_2017.zip extraido
dfp_cia_aberta_2018.zip extraido
dfp_cia_aberta_2019.zip extraido
dfp_cia_aberta_2020.zip extraido
dfp_cia_aberta_2021.zip extraido
dfp_cia_aberta_2022.zip extraido
itr_cia_aberta_2020.zip extraido
itr_cia_aberta_2021.zip extraido
itr_cia_aberta_2022.zip extraido
fca_cia_aberta_2022.zip extraido
fre_cia_aberta_2019.zip extraido
fre_cia_aberta_2020.zip extraido
fre_cia_aberta_2021.zip extraido
fre_cia_aberta_2022.zip extraido
Pronto


In [13]:
# concatena os arquivos baixados do ano inicial até o final
processa_dados(year_0, year, year_0_tri)

#deleta pasta temporaria se existir
try:
    shutil.rmtree(f'./Data/temp')
except:
    pass

Processando dados
DFP BPA_con processado
DFP BPA_ind processado
DFP BPP_con processado
DFP BPP_ind processado
DFP DRE_con processado
DFP DRE_ind processado
DFP DFC_MD_con processado
DFP DFC_MI_con processado
DFP DFC_MD_ind processado
DFP DFC_MI_ind processado
ITR BPA_con processado
ITR BPA_ind processado
ITR BPP_con processado
ITR BPP_ind processado
ITR DRE_con processado
ITR DRE_ind processado
ITR DFC_MD_con processado
ITR DFC_MI_con processado
ITR DFC_MD_ind processado
ITR DFC_MI_ind processado
FRE processado
FCA processado
Arquivos processados


## Fim do processamento dos dados

In [142]:
# cria um dataframe com todas as empresas listadas, tipo de balanço, ticker e ações em circulação Já vem bastante limpo.
lista_empresas = cria_lista_empresas(year_0, year, year_0_tri)
#lista_empresas.to_excel("teste.xlsx")
lista_empresas

,DENOM_CIA,CD_CVM,tipo,TICKER,Quantidade_Total_Acoes_Circulacao
00.000.000/0001-91,BCO BRASIL S.A.,1023,con,BBAS,1.420731e+09
00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,con,BSLI,4.195216e+07
00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,con,ELET,5.997421e+08
00.070.698/0001-11,CIA ENERGETICA DE BRASILIA,14451,con,CEBR,1.430870e+07
00.242.184/0001-04,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",26069,con,ARML,1.718094e+08
...,...,...,...,...,...
94.813.102/0001-70,TRÊS TENTOS AGROINDUSTRIAL S/A,25950,ind,TTEN,1.123436e+08
95.426.862/0001-97,EXCELSIOR ALIMENTOS S.A.,1570,ind,BAUH,1.508636e+06
96.418.264/0218-02,LOJAS QUERO QUERO S.A.,25038,con,LJQQ,1.789674e+08
97.191.902/0001-94,CONSERVAS ODERICH S.A.,4693,con,ODER,1.979520e+05


In [ ]:
# calcular indicadores e colocar cotação (adiar ao maximo pq demora mto tempo pra pegar)

In [71]:
lista_empresas = Coloca_lucro(lista_empresas)
lista_empresas

,DENOM_CIA,CD_CVM,tipo,TICKER,Quantidade_Total_Acoes_Circulacao,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,TTM
CNPJ,,,,,,,,,,,,,,,,,,
00.000.000/0001-91,BCO BRASIL S.A.,1023,con,BBAS,1.420731e+09,1.133034e+10,1.273691e+10,1.143820e+10,1.128883e+10,1.334350e+10,1.579804e+10,8.659577e+09,1.227530e+10,1.508610e+10,1.888832e+10,1.329288e+10,1.972287e+10,NaN
00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,con,BSLI,4.195216e+07,2.621840e+08,1.289360e+08,1.956440e+08,2.112250e+08,2.268760e+08,1.510270e+08,1.704430e+08,2.190700e+08,3.242110e+08,4.171460e+08,NaN,NaN,NaN
00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,con,ELET,5.997421e+08,2.552985e+09,3.762019e+09,-6.925652e+09,-6.291375e+09,-2.962502e+09,-1.495366e+10,3.513276e+09,-1.725691e+09,1.334758e+10,1.074384e+10,6.387313e+09,5.713633e+09,6.820869e+09
00.070.698/0001-11,CIA ENERGETICA DE BRASILIA,14451,con,CEBR,1.430870e+07,1.961700e+07,4.613200e+07,7.467900e+07,-7.516300e+07,-9.631900e+07,8.851900e+07,1.336930e+08,1.772330e+08,1.157100e+08,1.566290e+08,4.653780e+08,1.294407e+09,1.966290e+08
00.242.184/0001-04,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",26069,con,ARML,1.718094e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.806400e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91.669.747/0001-92,FINANSINOS S.A.- CREDITO FINANC E INVEST,6076,ind,FNCN,3.001010e+05,4.530000e+05,6.310000e+05,1.394000e+06,4.620000e+05,6.810000e+05,-4.539000e+06,-1.280000e+05,3.750000e+05,-2.323000e+06,-6.270000e+05,-2.088000e+06,-1.143000e+06,-1.336000e+06
92.749.225/0001-63,HERCULES S.A. FABRICA DE TALHERES,6629,ind,HETA,2.437500e+05,-1.658600e+07,-3.635700e+07,-3.136600e+07,-3.386600e+07,8.011700e+07,-6.738000e+06,3.045000e+06,-5.250800e+07,1.344000e+06,3.436000e+06,-1.720490e+08,4.430000e+05,-5.600000e+04
93.828.986/0001-73,CEMEPE INVESTIMENTOS S.A.,13854,ind,MAPT,3.569300e+04,-1.110000e+05,-3.780000e+05,-1.430000e+05,-1.330000e+05,-1.380000e+05,-2.090000e+05,-2.000000e+05,-1.410000e+05,-1.760000e+05,-1.630000e+05,-1.700000e+05,-1.780000e+05,-1.720000e+05


In [72]:
# coloca os indicadores  na lista
lista_empresas = Indicadores_lucro(lista_empresas)
lista_empresas

,DENOM_CIA,CD_CVM,tipo,TICKER,Quantidade_Total_Acoes_Circulacao,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,TTM,Sem prejuízo nos últimos 10 anos,Crescimento de 30% dos lucros nos últimos 10 anos
CNPJ,,,,,,,,,,,,,,,,,,,,
00.000.000/0001-91,BCO BRASIL S.A.,1023,con,BBAS,1.420731e+09,1.133034e+10,1.273691e+10,1.143820e+10,1.128883e+10,1.334350e+10,1.579804e+10,8.659577e+09,1.227530e+10,1.508610e+10,1.888832e+10,1.329288e+10,1.972287e+10,NaN,1.0,1.0
00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,con,BSLI,4.195216e+07,2.621840e+08,1.289360e+08,1.956440e+08,2.112250e+08,2.268760e+08,1.510270e+08,1.704430e+08,2.190700e+08,3.242110e+08,4.171460e+08,NaN,NaN,NaN,1.0,1.0
00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,con,ELET,5.997421e+08,2.552985e+09,3.762019e+09,-6.925652e+09,-6.291375e+09,-2.962502e+09,-1.495366e+10,3.513276e+09,-1.725691e+09,1.334758e+10,1.074384e+10,6.387313e+09,5.713633e+09,6.820869e+09,0.0,0.0
00.070.698/0001-11,CIA ENERGETICA DE BRASILIA,14451,con,CEBR,1.430870e+07,1.961700e+07,4.613200e+07,7.467900e+07,-7.516300e+07,-9.631900e+07,8.851900e+07,1.336930e+08,1.772330e+08,1.157100e+08,1.566290e+08,4.653780e+08,1.294407e+09,1.966290e+08,0.0,0.0
00.242.184/0001-04,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",26069,con,ARML,1.718094e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.806400e+07,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91.669.747/0001-92,FINANSINOS S.A.- CREDITO FINANC E INVEST,6076,ind,FNCN,3.001010e+05,4.530000e+05,6.310000e+05,1.394000e+06,4.620000e+05,6.810000e+05,-4.539000e+06,-1.280000e+05,3.750000e+05,-2.323000e+06,-6.270000e+05,-2.088000e+06,-1.143000e+06,-1.336000e+06,0.0,0.0
92.749.225/0001-63,HERCULES S.A. FABRICA DE TALHERES,6629,ind,HETA,2.437500e+05,-1.658600e+07,-3.635700e+07,-3.136600e+07,-3.386600e+07,8.011700e+07,-6.738000e+06,3.045000e+06,-5.250800e+07,1.344000e+06,3.436000e+06,-1.720490e+08,4.430000e+05,-5.600000e+04,0.0,0.0
93.828.986/0001-73,CEMEPE INVESTIMENTOS S.A.,13854,ind,MAPT,3.569300e+04,-1.110000e+05,-3.780000e+05,-1.430000e+05,-1.330000e+05,-1.380000e+05,-2.090000e+05,-2.000000e+05,-1.410000e+05,-1.760000e+05,-1.630000e+05,-1.700000e+05,-1.780000e+05,-1.720000e+05,0.0,0.0


In [73]:
lista_empresas.to_excel("teste.xlsx")

In [143]:
# Checar se faturamento é maior que 300 milhões
# acho que uma ideia melhor é pra todas as linhas do arquivo DRE adicionar uma coluna com trimestre e ano. ex: tri: 0, ano: 2021
# colocar isso no processamento do arquivo depois

def Coloca_faturamento(lista_empresas):
    tipo = "con"
    dre_dfp = pd.read_csv(f'./Data/processados/dfp_cia_aberta_DRE_{tipo}_{year_0}-{year}.csv', encoding="latin1")
    dre_dfp = dre_dfp[(dre_dfp["ORDEM_EXERC"] == "ÚLTIMO")]
    dre_dfp.set_index("CNPJ_CIA", inplace=True)

    # pega só o faturamento
    dre_dfp = dre_dfp[(dre_dfp.CD_CONTA == '3.01')].copy()

    # o ano é sempre o ano da data fim
    dre_dfp["Ano"] = dre_dfp.DT_FIM_EXERC.str[0:4]
    dre_dfp["Trimestre"] = 0

    dre_itr = pd.read_csv(f'./Data/processados/itr_cia_aberta_DRE_{tipo}_{year_0_tri}-{year}.csv', encoding="latin1")
    dre_itr = dre_itr[dre_itr["ORDEM_EXERC"] == "ÚLTIMO"]
    dre_itr.set_index("CNPJ_CIA", inplace=True)

    # pega só o faturamento
    dre_itr = dre_itr[(dre_itr.CD_CONTA == '3.01')].drop_duplicates().copy()

    # poe a coluna ano
    dre_itr.loc[:, "Ano"] = dre_itr.DT_REFER.str[0:4]

    # pra todos os CNPJ's
    for cnpj in lista_empresas[lista_empresas.tipo == tipo].index:
        # pega o mes de inicio do exercicio
        ini_exercicio = ini_exercicio = dre_dfp[(dre_dfp.index == cnpj)].sort_values(by=["DT_INI_EXERC", "DT_FIM_EXERC"])["DT_INI_EXERC"].str[5:7][0]

        # calcula o mes de inicio pra cada trimestre
        ini_tris = [int(ini_exercicio)+3*i for i in range(0,4)]

        # se o mes for maior que 12, diminui 12
        for i in range(len(ini_tris)):
            if ini_tris[i] > 12:
                ini_tris[i] -= 12

        # calcula os meses de fim de trimestre
        fim_tris = [i+2 for i in ini_tris]

        # transforma em string e preenche com 0 na frente
        fim_tris = [str(item).zfill(2) for item in fim_tris]
        ini_tris = [str(item).zfill(2) for item in ini_tris]
        # ex: ini_tris, fim_tris = ['04', '07', '10', '01'], ['06', '09', '12', '03']

        # poe uma coluna de nome pra cada trimestre
        for tri_ini, tri_fim, nome_tri in zip(ini_tris, fim_tris, ["1", "2", "3", "0"]):
            dre_itr.loc[(dre_itr.DT_INI_EXERC.str[5:7] == tri_ini) & 
                   (dre_itr.DT_FIM_EXERC.str[5:7] == tri_fim) & (dre_itr.index == cnpj), "Trimestre"] = nome_tri

    # tira tudo que não colocou trimestre, geralmente uns periodos aleatórios reportados
    dre_itr.dropna(subset=["Trimestre"], inplace=True)

    dre = pd.concat([dre_dfp, dre_itr], axis=0)
    
    
    
    ################################################################################################
    # faz a mesma coisa pro individual
    
    tipo = "ind"
    dre_dfp = pd.read_csv(f'./Data/processados/dfp_cia_aberta_DRE_{tipo}_{year_0}-{year}.csv', encoding="latin1")
    dre_dfp = dre_dfp[(dre_dfp["ORDEM_EXERC"] == "ÚLTIMO")]
    dre_dfp.set_index("CNPJ_CIA", inplace=True)

    # pega só o faturamento
    dre_dfp = dre_dfp[(dre_dfp.CD_CONTA == '3.01')].copy()

    # o ano é sempre o ano da data fim
    dre_dfp["Ano"] = dre_dfp.DT_FIM_EXERC.str[0:4]
    dre_dfp["Trimestre"] = 0

    dre_itr = pd.read_csv(f'./Data/processados/itr_cia_aberta_DRE_{tipo}_{year_0_tri}-{year}.csv', encoding="latin1")
    dre_itr = dre_itr[dre_itr["ORDEM_EXERC"] == "ÚLTIMO"]
    dre_itr.set_index("CNPJ_CIA", inplace=True)

    # pega só o faturamento
    dre_itr = dre_itr[(dre_itr.CD_CONTA == '3.01')].drop_duplicates().copy()

    # poe a coluna ano
    dre_itr.loc[:, "Ano"] = dre_itr.DT_REFER.str[0:4]

    # pra todos os CNPJ's
    for cnpj in lista_empresas[lista_empresas.tipo == tipo].index:
        # pega o mes de inicio do exercicio
        ini_exercicio = ini_exercicio = dre_dfp[(dre_dfp.index == cnpj)].sort_values(by=["DT_INI_EXERC", "DT_FIM_EXERC"])["DT_INI_EXERC"].str[5:7][0]

        # calcula o mes de inicio pra cada trimestre
        ini_tris = [int(ini_exercicio)+3*i for i in range(0,4)]

        # se o mes for maior que 12, diminui 12
        for i in range(len(ini_tris)):
            if ini_tris[i] > 12:
                ini_tris[i] -= 12

        # calcula os meses de fim de trimestre
        fim_tris = [i+2 for i in ini_tris]

        # transforma em string e preenche com 0 na frente
        fim_tris = [str(item).zfill(2) for item in fim_tris]
        ini_tris = [str(item).zfill(2) for item in ini_tris]
        # ex: ini_tris, fim_tris = ['04', '07', '10', '01'], ['06', '09', '12', '03']

        # poe uma coluna de nome pra cada trimestre
        for tri_ini, tri_fim, nome_tri in zip(ini_tris, fim_tris, ["1", "2", "3", "0"]):
            dre_itr.loc[(dre_itr.DT_INI_EXERC.str[5:7] == tri_ini) & 
                   (dre_itr.DT_FIM_EXERC.str[5:7] == tri_fim) & (dre_itr.index == cnpj), "Trimestre"] = nome_tri

    # tira tudo que não colocou trimestre, geralmente uns periodos aleatórios reportados
    dre_itr.dropna(subset=["Trimestre"], inplace=True)
    
    # junta o consolidado com o individual
    dre = pd.concat([dre, dre_dfp, dre_itr], axis=0)
    
    # dre está pronto. falta colocar o faturamento. a função que tinha funcionado antes está lá em baixo
    
    return(lista_empresas)
                                                              
Coloca_faturamento(lista_empresas)

ValueError: cannot reindex from a duplicate axis

In [ ]:
# coloca o faturamento na lista_empresas pras consolidadas
for cnpj in lista_empresas[lista_empresas.tipo == tipo].index:
        ult_valor = dre.loc[((dre.index == cnpj)), ["Ano", "Trimestre"]].sort_values(by=["Ano", "Trimestre"], ascending=False)
        if len(ult_valor) > 0:
            lista_empresas.loc[cnpj, "Faturamento"] = dre[(dre.index == cnpj) & (dre["Trimestre"] == ult_valor)]["Trimestre"].iloc[0]

In [119]:
# coloca o faturamento na lista_empresas pras consolidadas
for cnpj in lista_empresas[lista_empresas.tipo == tipo].index:
    ult_valor = dre.loc[((dre.index == cnpj)), ["Ano", "Trimestre"]].sort_values(by=["Ano", "Trimestre"], ascending=False)
    if len(ult_valor) > 0:
        lista_empresas.loc[cnpj, "Faturamento"] = dre[(dre.index == cnpj) & (dre["Trimestre"] == ult_valor)]["Trimestre"].iloc[0]

,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,Ano,Trimestre
CNPJ_CIA,,,,,,,,,,,,,,,,
00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01,Receitas da Intermediação Financeira,85143206.0,S,2010,0
00.000.208/0001-00,2010-12-31,2,BRB BCO DE BRASILIA S.A.,14206,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01,Receitas da Intermediação Financeira,1425290.0,S,2010,0
00.001.180/0001-26,2010-12-31,4,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01,Receita de Venda de Bens e/ou Serviços,29814652.0,S,2010,0
00.070.698/0001-11,2010-12-31,2,CIA ENERGETICA DE BRASILIA,14451,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01,Receita de Venda de Bens e/ou Serviços,1839320.0,S,2010,0
00.095.147/0001-02,2010-12-31,1,VBC ENERGIA SA,14540,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01,Receita de Venda de Bens e/ou Serviços,3074336.0,S,2010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96.418.264/0218-02,2022-03-31,1,LOJAS QUERO QUERO S.A.,25038,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2022-01-01,2022-03-31,3.01,Receita de Venda de Bens e/ou Serviços,540249.0,S,2022,1
96.418.264/0218-02,2022-06-30,1,LOJAS QUERO QUERO S.A.,25038,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2022-04-01,2022-06-30,3.01,Receita de Venda de Bens e/ou Serviços,556157.0,S,2022,2
97.191.902/0001-94,2022-03-31,1,CONSERVAS ODERICH S.A.,4693,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2022-01-01,2022-03-31,3.01,Receita de Venda de Bens e/ou Serviços,166992.0,S,2022,1


In [118]:
dre_itr = pd.read_csv(f'./Data/processados/itr_cia_aberta_DRE_{tipo}_{year_0_tri}-{year}.csv', encoding="latin1")
dre_itr = dre_itr[dre_itr["ORDEM_EXERC"] == "ÚLTIMO"]
dre_itr.set_index("CNPJ_CIA", inplace=True)

# pega só o faturamento
dre_itr = dre_itr[(dre_itr.CD_CONTA == '3.01')].drop_duplicates().copy()

# poe a coluna ano
dre_itr.loc[:, "Ano"] = dre_itr.DT_REFER.str[0:4]

# pra todos os CNPJ's
for cnpj in lista_empresas[lista_empresas.tipo == "con"].index:
    # pega o mes de inicio do exercicio
    ini_exercicio = ini_exercicio = dre_dfp[(dre_dfp.index == cnpj)].sort_values(by=["DT_INI_EXERC", "DT_FIM_EXERC"])["DT_INI_EXERC"].str[5:7][0]

    # calcula o mes de inicio pra cada trimestre
    ini_tris = [int(ini_exercicio)+3*i for i in range(0,4)]

    # se o mes for maior que 12, diminui 12
    for i in range(len(ini_tris)):
        if ini_tris[i] > 12:
            ini_tris[i] -= 12

    # calcula os meses de fim de trimestre
    fim_tris = [i+2 for i in ini_tris]

    # transforma em string e preenche com 0 na frente
    fim_tris = [str(item).zfill(2) for item in fim_tris]
    ini_tris = [str(item).zfill(2) for item in ini_tris]
    # ex: ini_tris, fim_tris = ['04', '07', '10', '01'], ['06', '09', '12', '03']
    
    # poe uma coluna de nome pra cada trimestre
    for tri_ini, tri_fim, nome_tri in zip(ini_tris, fim_tris, ["1", "2", "3", "0"]):
        dre_itr.loc[(dre_itr.DT_INI_EXERC.str[5:7] == tri_ini) & 
               (dre_itr.DT_FIM_EXERC.str[5:7] == tri_fim) & (dre_itr.index == cnpj), "Trimestre"] = nome_tri


dre_itr.dropna(subset=["Trimestre"], inplace=True)
dre_itr

In [112]:
cnpj = "97.837.181/0001-47"
ini_exercicio = ini_exercicio = dre_dfp[(dre_dfp.index == cnpj)].sort_values(by=["DT_INI_EXERC", "DT_FIM_EXERC"])["DT_INI_EXERC"].str[5:7][0]

# calcula o mes de inicio pra cada trimestre
ini_tris = [int(ini_exercicio)+3*i for i in range(0,4)]

# se o mes for maior que 12, diminui 12
for i in range(len(ini_tris)):
    if ini_tris[i] > 12:
        ini_tris[i] -= 12

# calcula os meses de fim de trimestre
fim_tris = [i+2 for i in ini_tris]

# transforma em string e preenche com 0 na frente
fim_tris = [str(item).zfill(2) for item in fim_tris]
ini_tris = [str(item).zfill(2) for item in ini_tris]
#ini_exercicio = dre_dfp[(dre_dfp.index == cnpj)].sort_values(by=["DT_INI_EXERC", "DT_FIM_EXERC"])["DT_INI_EXERC"].str[5:7][0]
#ini_exercicio
print(ini_tris, fim_tris)

tri_ini = ini_tris[0]
tri_fim = fim_tris[0]

dre_itr.loc[(dre_itr.DT_INI_EXERC.str[5:7] == tri_ini) & 
               (dre_itr.DT_FIM_EXERC.str[5:7] == tri_fim) & (dre_itr.index == cnpj), "Trimestre"] = "bunda"
dre_itr
#dre_itr.loc[((dre_itr.DT_INI_EXERC.str[5:7] == tri_ini)), "Trimestre"]

['01', '04', '07', '10'] ['03', '06', '09', '12']


,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,Trimestre
CNPJ_CIA,,,,,,,,,,,,,,,
00.001.180/0001-26,2020-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2020-01-01,2020-03-31,3.01,Receita de Venda de Bens e/ou Serviços,6955636.0,S,NaN
00.001.180/0001-26,2020-06-30,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2020-01-01,2020-06-30,3.01,Receita de Venda de Bens e/ou Serviços,18053226.0,S,NaN
00.001.180/0001-26,2020-06-30,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2020-04-01,2020-06-30,3.01,Receita de Venda de Bens e/ou Serviços,11097590.0,S,NaN
00.001.180/0001-26,2020-09-30,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2020-01-01,2020-09-30,3.01,Receita de Venda de Bens e/ou Serviços,25484442.0,S,NaN
00.001.180/0001-26,2020-09-30,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2020-07-01,2020-09-30,3.01,Receita de Venda de Bens e/ou Serviços,7431216.0,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96.418.264/0218-02,2022-06-30,1,LOJAS QUERO QUERO S.A.,25038,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2022-04-01,2022-06-30,3.01,Receita de Venda de Bens e/ou Serviços,556157.0,S,NaN
97.191.902/0001-94,2022-03-31,1,CONSERVAS ODERICH S.A.,4693,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2022-01-01,2022-03-31,3.01,Receita de Venda de Bens e/ou Serviços,166992.0,S,NaN
97.837.181/0001-47,2022-03-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2022-01-01,2022-03-31,3.01,Receita de Venda de Bens e/ou Serviços,2131002.0,S,bunda


In [ ]:

# multiplica o valor por 1000 onde a escala é 1000
dre.loc[(dre.ESCALA_MOEDA == "MIL"), "VL_CONTA"] *= 1000



#cria coluna trimestre e ano
for ano in range(year_0, year+1):
    dre.loc[((dre.DT_INI_EXERC.str.contains(f"{ano}-")) & 
               (dre.DT_FIM_EXERC.str.contains(f"{ano}-"))), "Ano"] = ano
    for tri_ini, tri_fim, nome_tri in zip(["01", "04", "07", "01"], ["03", "06", "09", "12"], ["1", "2", "3", "0"]):
        dre.loc[((dre.DT_INI_EXERC == (f"{ano}-{tri_ini}-01")) & 
               (dre.DT_FIM_EXERC.str.contains(f"{ano}-{tri_fim}-"))), "Trimestre"] = nome_tri


dre.dropna(subset=["Trimestre"], inplace=True)

# coloca o faturamento na lista_empresas pras consolidadas
for cnpj in lista_empresas.index:
    ult_valor = dre.loc[((dre.index == cnpj)), ["Ano", "Trimestre"]].sort_values(by=["Ano", "Trimestre"], ascending=False)
    if len(ult_valor) > 0:
        lista_empresas.loc[cnpj, "Faturamento"] = dre[(dre.index == cnpj) & (dre["Trimestre"] == ult_valor["Trimestre"].iloc[0])


In [65]:
# o for pra colocar trimestres dá problema quando a empresa não começa o ano em janeiro. (agro3, caml3, etc.)
# precisa arrumar pra coloca_lucro tbm. Talvez usar datetime pra medir 1 ano

In [ ]:
# Futuro modulo de gerar relatório
empresa = escolhe_empresa(lista_empresas)
cnpj_selecionado = empresa.index[0]
tipo_empresa_selecionada = empresa["tipo"].iloc[0]
cnpj_selecionado, tipo_empresa_selecionada